In [1]:
#ejecutar solo la primera vez para instalar las dependencias

#!python ./get-pip.py
#!pip install transformers
#!pip install torch
#!pip install pandas
#!pip install fastparquet
#!pip install --upgrade pip
#!pip install accelerate>=0.20.1
#!pip install transformers[torch]
#!pip install scikit-learn
#!pip install accelerate -U
#!pip install tensorflow

In [2]:
import pandas as pd
from transformers import BertTokenizer, BertForQuestionAnswering, TrainingArguments, Trainer
import torch
from sklearn.model_selection import train_test_split

In [3]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('data.csv')

# Extraer las preguntas y respuestas del DataFrame
questions = df['query'].tolist()
answers = df['positive'].tolist()

# Tokenización de las preguntas
tokenizer = BertTokenizer.from_pretrained('MMG/bert-base-spanish-wwm-cased-finetuned-squad2-es')

encodings = tokenizer(questions,answers, truncation=True, padding=True)

# Carga del modelo BERT basado en PyTorch
model = BertForQuestionAnswering.from_pretrained('MMG/bert-base-spanish-wwm-cased-finetuned-squad2-es')

In [4]:
print(questions)
print(answers)

['está bien un poco de cafeína durante el embarazo', '¿Qué fruta es originaria de Australia?', '¿Qué tan grande es el ejército canadiense?', 'tipos de árboles frutales', 'cuántas calorías al día se pierden amamantando', 'donde esta harrison city pa', 'cuántos representantes tiene texas', 'Principales características climáticas de las selvas tropicales.', 'síntomas del cáncer de páncreas', 'alimentos que ayudarán a reducir el azúcar en sangre', 'qué condado es williamsport, pa', '¿Cuáles son algunos alimentos que evitan las mujeres embarazadas?', '¿Cuánto tiempo tarda la sangre en reponer una pérdida de sangre?', '¿Cuál es el género del sauce llorón?', '¿Qué hizo Rudolf Virchow por las células?', 'tabla de colesterol bueno y malo', 'números a letras en el alfabeto', 'es el colesterol bueno realmente bueno', '¿Puedes quedar embarazada el primer día de tu período?', 'qué productos químicos usar en el cardo canadiense', 'usar comas correctamente en oraciones', 'que impulsó la revolución in

In [5]:
start_positions = []
end_positions = []

for question, answer in zip(questions, answers):
    inputs = tokenizer(question, answer, truncation=True, padding=True, return_tensors="pt")

    output = model(**inputs)
    start_logits = output.start_logits
    end_logits = output.end_logits

    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    start_positions.append(start_index.item())
    end_positions.append(end_index.item())


In [6]:
class CustomQADataset(torch.utils.data.Dataset):
    def __init__(self, questions, answers, start_positions, end_positions, tokenizer, max_length):
        self.questions = questions
        self.answers = answers
        self.start_positions = start_positions
        self.end_positions = end_positions
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        encoding = self.tokenizer(
            question,
            answer,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor(self.start_positions[idx]),
            'end_positions': torch.tensor(self.end_positions[idx])
        }

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y evaluación
train_questions, eval_questions, train_answers, eval_answers, train_start_positions, eval_start_positions, train_end_positions, eval_end_positions = train_test_split(
    questions, answers, start_positions, end_positions, test_size=0.2, random_state=42)

train_dataset = CustomQADataset(train_questions, train_answers, train_start_positions, train_end_positions, tokenizer, max_length=512)

eval_dataset = CustomQADataset(eval_questions, eval_answers, eval_start_positions, eval_end_positions, tokenizer, max_length=512)

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    per_device_eval_batch_size=8,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
trainer.evaluate()